In [30]:
import nltk
import os
import re
import itertools
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import codecs
from textblob import TextBlob
import pandas as pd
import re
%matplotlib inline
%load_ext autoreload

def removeObservations(test_str):
    # removes all text within parantheses
    # these are usually observations by the transcriber
    # and thus should not be considered in content analysis
    # code from http://stackoverflow.com/questions/14596884/remove-text-between-and-in-python
    ret = ''
    skip1c = 0
    skip2c = 0
    for i in test_str:
        if i == '[':
            skip1c += 1
        elif i == '(':
            skip2c += 1
        elif i == ']' and skip1c > 0:
            skip1c -= 1
        elif i == ')'and skip2c > 0:
            skip2c -= 1
        elif skip1c == 0 and skip2c == 0:
            ret += i
    return ret

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
# Open and read transcripts in NLTK
f1=codecs.open('free_group/teamA_part1_divergent.txt', 'r', encoding='utf-8')
f2=codecs.open('free_group/teamA_part2_convergent.txt', 'r', encoding='utf-8')
f1Temp = codecs.open('tempASCII', 'w', encoding='ascii', errors='ignore')
f2Temp = codecs.open('tempASCII', 'w', encoding='ascii', errors='ignore')
f1Temp.write('free_group/teamA_part1_divergent_bak.txt')
f2Temp.write('free_group/teamA_part1_convergent_bak.txt')
divergent_raw = f1.read()
convergent_raw = f2.read()
regex = re.compile(".*?\[(.*?)\]")
divergent = removeObservations(divergent_raw)
convergent = removeObservations(convergent_raw)

In [26]:
# Extract Noun phrases using TextBlob
blobDivergent = TextBlob(divergent)
blobConvergent = TextBlob(convergent)
nounDiv = blobDivergent.np_counts
nounConv = blobConvergent.np_counts
dfnp1 = pd.DataFrame(columns=['Noun phrases in Divergent phase', 'Frequencies'])
dfnp2 = pd.DataFrame(columns=['Noun phrases in Convergent phase', 'Frequencies'])
u=1
for i in nounDiv:
    dfnp1.loc[u] = [i, nounDiv[i]]
    u+=1

u=1
for i in nounConv:
    dfnp2.loc[u] = [i, nounConv[i]]
    u+=1
    
print("The total number of noun phrases in the divergent phase:", len(nounDiv))
dfnp1.head(15)

The total number of noun phrases in the divergent phase: 222


,Noun phrases in Divergent phase,Frequencies
1,giant man,1
2,marble run,2
3,big magnetic ball,1
4,marble run-type thing,1
5,magnetic part,1
6,person draws,1
7,old school,1
8,you’ve,1
9,magnetic thing,1
10,rube goldberg,1


In [27]:
print("The total number of noun phrases in the convergent phase:", len(nounConv))
dfnp2.head(15)

The total number of noun phrases in the convergent phase: 104


,Noun phrases in Convergent phase,Frequencies
1,good drawings,1
2,dinosaur roar,2
3,don’t wan na implement,1
4,mechanical motion,1
5,commercial use,1
6,final concept,6
7,gorilla’s arm,1
8,especially,1
9,they’ll,1
10,single,1


In [28]:
# Common noun phrases in both phases
common = {}
notcommon = {}
dfc = pd.DataFrame(columns=["Common noun phrases", "Divergent", "Convergent"])
u=1
for i in nounDiv:
    if i in nounConv:
        common[i] = min(nounDiv[i], nounConv[i])
        notcommon[i] = (nounDiv[i], nounConv[i])
        dfc.loc[u] = [i, nounDiv[i], nounConv[i]]
        u+=1
dfc

,Common noun phrases,Divergent,Convergent
1,i’m,3,5
2,that’s,10,1
3,one’s gon na,1,1
4,okay,10,2
5,good idea,3,1
6,here’s,1,1
7,whole thing,1,1
8,it’s,8,2
9,say,1,1
10,have,2,2


In [32]:
# Noun phrases that can be removed:
to_be_removed = [u'okay', u'yeah']
keys_to_delete = []
for key in common.keys():
    if key in to_be_removed:
        keys_to_delete.append(key)
for key in keys_to_delete:
    del common[key] # could not have deleted item from iterable 
                    # from within the loop
ccommon=common.keys()

In [ ]:
# count co-occurrences of common noun phrases in sentences
def occurrences(source, terms):
    ALL_sentences=sent_tokenize(source)
    combinations_terms - list(itertools.combinations(terms,2))
    n = len(combinations_terms)
    occurlist = []
    for i in range(1, n):
        for j in ALL_sentences:
            temp = list(combinations_terms[i])
            out=re.compile(str(temp[0]) 